In [41]:
import sys
sys.path.append('../../src')

In [42]:
from auto_prompt.llm.openai import OpenAICompletionEngine

In [169]:
sql_code = 'selct category, count(*) from mytable where year >= 2022 group by category order by count(*) limit 10'

original_prompt = 'Given a sql code, explain the code. This is the sql code: ```{sql_code}```'
original_prompt = 'Given a SQL code, provide a concise, step-by-step explanation of the code without making references to actual pieces of code. Your explanation should be comprehensive and accessible to individuals with little knowledge of data analytics. Here is the SQL code: ```{sql_code}```'
original_prompt = 'Given a SQL code, generate a concise, itemized list of action steps interpreting what the code is doing. Avoid using technical jargon and make your explanation comprehensive for individuals with a minimal understanding of data analytics. Do not reference actual pieces of code in your explanation. Here is the SQL code: ```{sql_code}```'
original_propmt = 'Given a SQL code, generate a point-by-point breakdown of actions that are being performed in the code. Make your explanation easy-to-grab for those with little to no understanding of data analytics. Your task is to convey what the code is doing without using technical terms or citing the actual snippets of the code. Provide each step as a separate numbered line. Remember clarity, brevity and understanding are key. Please do not include any supplementary text before, after or within the list. Here is the SQL code: ```{sql_code}```'
original_prompt = 'Given a SQL code, generate a concise, step-by-step non-technical interpretation of what the code aims to accomplish. Your explanation should help someone with limited data analytics experience to understand the purpose of the code. Ensure your explanation is organized as an itemized list, no introduction or conclusion is needed. Additionally, refrain from referencing actual pieces of code from the SQL query. Here is the SQL code: ```{sql_code}```'
original_prompt = 'Given a SQL code, generate a structured, point by point non-technical interpretation that outlines the purpose and operations in the SQL command. Strive for brevity and clarity in your explanations, and avoid direct referencing to SQL terms. Bear in mind that your audience may not have extensive knowledge in data analytics or SQL hence the interpretations should be easy to follow and understand at a basic level. Your interpretation should come in the form of an itemized list without any preamble or conclusion. Here is the SQL code: ```{sql_code}```'
original_prompt = 'Given a SQL command, outline a clear, point-by-point interpretation that conveys the intention and the actions in the SQL command in non-technical language. Aim for simplicity and conciseness in your explanations, with an emphasis on stripping SQL references to make it understandable to a novice. Explain the steps as bullet points, with no introductory or concluding statements. Your audience may not possess a deep understanding of data analytics or SQL language, so make sure your explanations are comprehensive but straightforward. Here is the SQL command: ```{sql_code}```'
original_prompt = """
Given a SQL command, provide a clear, step-by-step interpretation of the command that explains the intention and actions in everyday language. Your target audience may not know much, if anything, about data analytics or SQL language.

Set your explanations as a list of bullet points, focusing only on each step of the operation and not on the actual code. Avoid introductory or concluding statements. Remain concise while also ensuring you're comprehensive in your explanation to make the SQL command understandable for a beginner. Follow this format: ```{sql_code}```.
- Step 1: ...
- Step 2: ...
- Step 3: ...
- ...
Here is the SQL command: ```{sql_code}```.
"""

original_prompt = """
Given a SQL command, provide a clear, step-by-step interpretation of the command that breaks down the intention and actions in ordinary language. Your audience may not have much, if any, background in data analytics or SQL language. 

Make your explanations a sequence of steps focused entirely on each operation's phase, not on the actual code. Keep away from beginning or concluding statements. Stay succinct while also ensuring that your explanation is comprehensive to make the SQL command understandable for a novice. Follow the format: 

- Step 1: <Step 1 explanation...>
- Step 2: <Step 2 explanation...>
- Step 3: <Step 3 explanation...>
- ...

Here is the SQL command: ```{sql_code}```. 

Be sure to leave out any final thoughts or conclusions about the command as a whole. Stick to explaining the steps only.
"""

original_prompt = """

Given a SQL command, your task is to translate it into a concise sequence of steps that explain the command's intentions and actions in layman's terms. It is important to keep in mind that your audience may not have prior knowledge about data analytics or SQL programming language. 

Your explanations should provide a clear step-by-step guide focusing on each operation's phase without referring directly to the actual code. Aim for brevity while ensuring that your explanatory steps provide a broad understanding of the SQL command to a beginner. Write accordingly:

- Step 1: <Insert your explanation>
- Step 2: <Insert your explanation>
- Step 3: <Insert your explanation>
- ...

Please bypass any introduction or conclusion about the overall command. Concentrate solely on explaining each specific step.

Here is the SQL command you need to explain: ```{sql_code}```. 

Do not incorporate any concluding thoughts or interpretations about the entire command. Stick to explaining the individual steps alone. Make it a point to not refer to any SQL code or technical jargon. Your explanations should be as simple as possible to appropriately guide a novice.
"""

completion_engine = OpenAICompletionEngine()

completion_engine.insert_message('system', 'you are a sql expert.')
completion_engine.insert_message('user', original_prompt.format(sql_code=sql_code))

In [170]:
result = completion_engine.get_response()
explanation = result.choices[0].message.content

In [171]:
print(explanation)

- Step 1: Look into a filing cabinet labeled "mytable."
- Step 2: Gather all the files in it.
- Step 3: Ignore files from before the year 2022.
- Step 4: Separate the remaining files into piles based on their category.
- Step 5: Count the number of files in each category pile.
- Step 6: Arrange the piles from largest to smallest according to the number of files in them.
- Step 7: Pick the top 10 biggest piles.
- Step 8: Record the name of each category and the size of its pile.


In [172]:
scoring_criteria = """
- The explanation should be concise.
- The explanation should purely be a list of steps, with no words before or after.
- The explanation does not need to make references to actual pieces of code. Just explain the steps.
- The explanation should be comprehensive and not assume much baseline knowledge of data analytics.
"""

In [173]:
scorer_engine = OpenAICompletionEngine()
scorer_engine.insert_message('system', 
    """
    You are a data analyst who is an expert at scoring explanations of SQL Code.
    """
)
scorer_engine.insert_message('user', 
    f"""
    You will be given a sql query, an explanation of the query and a set of criteria to evaluate the explanation.
    
    This is the sql query:
    ```
    {sql_code}
    ```
    
    This is the explanation:
    ```
    {explanation}
    ```
    
    This is the evaluation critera:
    ```
    {scoring_criteria}
    ```
    
    Your task is to evaluate the query explanation based on the criteria by giving it a score from 1 - 10.
    Just return the score as a single float number and do not return anything else.
    
    Score:
    ```
    """
)

In [174]:
score = scorer_engine.get_response().choices[0].message.content

In [175]:
score

'8.5'

In [176]:
examples = f"""
input: {sql_code}

response: {explanation}

score: {score}
"""

In [177]:
completion_engine = OpenAICompletionEngine()

completion_engine.insert_message('system',
    """
    You are a prompt engineer capable of improving a given prompt based on \
    feedback of the prompt's performance
    """
)
completion_engine.insert_message('user', 
    f"""
    You will first be provided with a previously used prompt. \
    Then, you will be given examples of (input, response, score) where the prompt was used to \
    generate the response from the input. \
    Separately, the quality of each response was given a score, \
    and the examples provied are the ones with the lowest scores.
    You will be given the description of the scoring criteria as context. \
    Based on these examples, you will then have to edit the prompt so that the score can improve.
    
    This is the original prompt:
    ```
    {original_prompt}
    ```
    
    This is the description of the scoring criteria used to generate the score of responses:
    ```
    {scoring_criteria}
    ```
    
    These are the examples of (input, response, score):
    ```
    {examples}
    ```
    
    Now, edit the original prompt to create a new prompt such that when the new prompt \
    is used on the example inputs above, the score of the response based on \
    the scoring criteria can be expected to improve.
    
    New Prompt:
    ```
    """
)

In [178]:
new_prompt = completion_engine.get_response().choices[0].message.content

In [179]:
print(original_prompt)
print()
print(new_prompt)



Given a SQL command, your task is to translate it into a concise sequence of steps that explain the command's intentions and actions in layman's terms. It is important to keep in mind that your audience may not have prior knowledge about data analytics or SQL programming language. 

Your explanations should provide a clear step-by-step guide focusing on each operation's phase without referring directly to the actual code. Aim for brevity while ensuring that your explanatory steps provide a broad understanding of the SQL command to a beginner. Write accordingly:

- Step 1: <Insert your explanation>
- Step 2: <Insert your explanation>
- Step 3: <Insert your explanation>
- ...

Please bypass any introduction or conclusion about the overall command. Concentrate solely on explaining each specific step.

Here is the SQL command you need to explain: ```{sql_code}```. 

Do not incorporate any concluding thoughts or interpretations about the entire command. Stick to explaining the individual 